# SICP中的编程技艺 1.2

# 程序及其生成的过程

[1.2 Procedures and the Processes They Generate](https://mitp-content-server.mit.edu/books/content/sectbyfn/books_pres_0/6515/sicp.zip/full-text/book/book-Z-H-11.html#%_sec_1.2)

## 线性递归和迭代

### 计算阶乘

$n! = 1 \times 2 \times 3 \times \cdot \cdot \cdot n$

或者

$n! =
\begin{cases} 
1 & \text{if } n = 0, \\
n \cdot (n-1)! & \text{if } n > 0.
\end{cases}$

我们先用递归式表达

In [1]:
(define (factorial n)
  (if (= n 0)
      1
      (* n (factorial (- n 1)))))

#<void>

In [2]:
(factorial 6)

720

形式基本上和数学公式对应。

In [4]:
(trace factorial)
(factorial 6)

|(factorial 6)
| (factorial 5)
| |(factorial 4)
| | (factorial 3)
| | |(factorial 2)
| | | (factorial 1)
| | | 1
| | |2
| | 6
| |24
| 120
|720


720

程序的执行是有"形状"的，这是线性递归的形状。

再看如何迭代，在有 `for` 语言的程序中，这非常容易：

```js
function factorial(n) {
  let product = 1
  for (let i = 1; i <= n; i++) {
    product *= i
  }
  return product
}
```

那如何使用函数表达迭代呢？ 有个模式， 就是把 累计值 `product`，和循环中用到的变量统统定义成函数参数。

In [6]:
(define (fact-iter product i n)  ;; 累计值, 循环变量， 循环结束变量
  (if (> i n)
      product
      (fact-iter (* product i)    ;; product *= i
                 (+ i 1)          ;; i++
                 n)))

#<void>

In [8]:
(fact-iter 1 1 6)

720

记住这个模板，它就是 for 的函数式版本，这和我们刚学编程时，第一次"背" `for (int i = 0; i< n; i++)` 没有什么本质区别，只是此刻我们忘记了当时的愚钝。

使用块作用域所它整合起来

In [9]:
(define (factorial n)
  (define (factorial-iter product i n)
    (if (> i n)
        product
        (factorial-iter (* product i)
                        (+ i 1)
                        n)))
  (factorial-iter 1 1 n))

#<void>

In [10]:
(factorial 8)

40320

In [13]:
(trace factorial)

(factorial)

In [14]:
(factorial 8)

|(factorial 8)
|40320


40320

需要 trace 一下执行流程

In [15]:
(define (factorial n)
  (trace-define (factorial-iter product i n)  ;; 使用 trace-define 定义
    (if (> i n)
        product
        (factorial-iter (* product i)
                        (+ i 1)
                        n)))
  (factorial-iter 1 1 n))

#<void>

In [16]:
(factorial 8)

|(factorial-iter 1 1 8)
|(factorial-iter 1 2 8)
|(factorial-iter 2 3 8)
|(factorial-iter 6 4 8)
|(factorial-iter 24 5 8)
|(factorial-iter 120 6 8)
|(factorial-iter 720 7 8)
|(factorial-iter 5040 8 8)
|(factorial-iter 40320 9 8)
|40320


40320

这是迭代的形态。

对于当前程序，看到有2个变量在弯化，最后一个变量不变，因此可以简化。

In [1]:
(define (factorial n)
  (define (factorial-iter product i)
    (if (> i n)  ;; 利用块作用域，能取到外层 n
        product
        (factorial-iter (* product i)
                        (+ i 1))))
  (factorial-iter 1 1))

#<void>

In [2]:
(factorial 8)

40320

factorial-iter函数调用了自身，所以应该是递归的，但是执行流程确是迭代的，所以我们要区分 **递归程序和递归过程**； 

程序可以使用递归的方式编写，在`Scheme`中经常如此，得益于尾递归的，执行过程是迭代的，不会消耗栈空间。

    在这一点上，熟悉其他提供特殊迭代构造的语言的读者，例如 while 或 for 循环，可能会想知道 Scheme 中是否需要类似的构造。这样的构造是不必要的；在 Scheme 中，迭代通过递归表达得更清晰和简洁。递归更为通用，并且消除了许多其他语言的迭代构造所需的变量赋值，从而产生更可靠且更易于遵循的代码。有些递归本质上就是迭代，并且作为迭代执行；第 3.2 节对此有更多的讨论。然而，通常没有必要做出区分。相反，应该集中精力编写清晰、简洁和正确的程序。
        —— 《The Scheme Programming Language》

[Section 2.8. Simple Recursion](https://www.scheme.com/tspl4/start.html#./start:h8)

## 尾递归

求列表长度

In [2]:
(define length
  (lambda (ls)
    (if (null? ls)
        0
        (+ (length (cdr ls)) 1))))

#<void>

In [4]:
(length '(1 2 3 4))

4

In [5]:
(trace length)

(length)

In [6]:
(length '(1 2 3 4))

|(length (1 2 3 4))
| (length (2 3 4))
| |(length (3 4))
| | (length (4))
| | |(length ())
| | |0
| | 1
| |2
| 3
|4


4

按上述方法改写

### 书中的例子

In [1]:
inc

Evaluation Error: variable inc is not bound

In [2]:
(define (inc x) (+ x 1))

#<void>

In [3]:
(inc 1)

2

In [9]:
(define (dec x) (- x 1))

#<void>

In [10]:
(dec 10)

9

来自于书中的例子，第一次接触时，感觉以这种方式思考加法，有趣。

In [18]:
(define (add a b )
  (if (= a 0)
      b
      (inc (add (dec a) b))))

#<void>

In [19]:
(add 5 6)

11

In [20]:
(trace add)

(add)

In [21]:
(add  5 6)

|(add 5 6)
| (add 4 6)
| |(add 3 6)
| | (add 2 6)
| | |(add 1 6)
| | | (add 0 6)
| | | 6
| | |7
| | 8
| |9
| 10
|11


11

In [26]:
(define (add a b)
  (if (= a 0)
      b
      (add (dec a) (inc b)))) ; 这句不同。

#<void>

In [23]:
(add 5 6)

11

In [24]:
(trace add)

(add)

In [25]:
(add 5 6)

|(add 5 6)
|(add 4 7)
|(add 3 8)
|(add 2 9)
|(add 1 10)
|(add 0 11)
|11


11

## 斐波那契数列

```scheme
0 1 1 2 3 5 8 13
```

每个值都是前两个值之和。

In [4]:
(define (fib n)
  (if (= n 0) 0
      (if (= n 1) 1
          (+
            (fib (- n 2))
            (fib (- n 1))))))

#<void>

In [5]:
(fib 7)

13

多个 if 可以用 cond 简化

In [8]:
(define (fib n)
  (cond ((= n 0) 0)
        ((= n 1) 1)
        (else
         (+
          (fib (- n 2))
          (fib (- n 1))))))

#<void>

In [9]:
(fib 8)

21

`cond` 会被转成 `if` 实现

In [11]:
(expand '(cond ((= 0 0) 0)
               ((= n 1) 1)
               (else
                'some_thing_else)))

(if (($primitive 2 =) 0 0) 0 (if (($primitive 2 =) n 1) 1 (quote some_thing_else)))

回到本小节主题，递归。

In [12]:
(trace fib)

(fib)

In [14]:
(fib 5)

|(fib 5)
| (fib 3)
| |(fib 1)
| |1
| |(fib 2)
| | (fib 0)
| | 0
| | (fib 1)
| | 1
| |1
| 2
| (fib 4)
| |(fib 2)
| | (fib 0)
| | 0
| | (fib 1)
| | 1
| |1
| |(fib 3)
| | (fib 1)
| | 1
| | (fib 2)
| | |(fib 0)
| | |0
| | |(fib 1)
| | |1
| | 1
| |2
| 3
|5


5

可以看到，这个递归的图就不是线性性的，它是指数级复杂度的。

fibs序列为:

```scheme
1 1 2 3 5 8 13 ... fibs(n)
```

它应该比

```scheme
1 2 4 8 16 32 ... 2^n
```

要小。

它应该比

```scheme
1 1 2 2 4 4 8 8 16 2^(n/2) ;;每一项都是前前项的2倍。
```

要大。

$ 2^{n/2} = \sqrt2^{n}$

所以

$ \sqrt2^{n} < fibs(n) < 2^{n} $

## 通项

这一步其实和编程关系不大，只是为了把题稍微解完整。

$ F(n) = F(n - 2) + F(n - 1) $

令 $ F(n) = r^{n} $

$ r^{n} = r^{n - 2} + r^{n - 1} $ 两边除以 $ r^{n - 2} $ 得

$ r^{2} = 1 + r $ 解得

$ r1 = (1 + \sqrt5) / 2 $  

$ r2 = (1 - \sqrt5) / 2 $

为了方便表示，令 $ u = r1, v = r2 $

通解为：

$ F(n) = Au^n + Bv^n $

$ F(0) = 0 $

$ F(1) = 1 $

代入得

$ A + B = 0 $

$ Au + Bv = 1 $

解得

$ A = 1 / \sqrt5 = 1 / \sqrt5 $

$ B = - (1 / \sqrt5) $

得到结果：

$ F(n) = \frac {u^{n} - v^{n}} {\sqrt5} $

其中

$ u = \frac{1 + \sqrt5} {2} $

$ v = \frac{1 - \sqrt5} {2} $

$ v \approx -0.6 $, 当n比较大时， $ v^{n} $ 很接近于0 是个负数。

因此可以使用 $ F(n) = (\frac{1+\sqrt5}{2})^{n} / \sqrt5 $ 画图

$ \frac{1+\sqrt5}{2} \approx 1.62  $

In [1]:
%%html

<iframe src="https://www.desmos.com/calculator/0s3z6rnas9?embed" width="500" height="500" style="border: 1px solid #ccc" frameborder=0></iframe>

[查看图表](https://www.desmos.com/calculator/631zig72ba)

其中

- 红线为 $ y=2^{n} $
- 绿线为 $ fib(n) \approx \sqrt5 \times 1.62^{n} $ 
- 蓝色为 $y=\sqrt2^{n} \approx 1.414^{n} $